In [1]:
import tweepy
import pandas as pd
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [2]:
API_KEY = "gqTUfjzorXK9mWPczSJmngovs"
API_secret_key = "g5BYesGeL875YhZH1bmM1pzw7sadBBdpQ0FTjntW6cfBbWaSk5"
Access_token = "1336583821773459459-x1plmJQwCWDDPc1OpcvTVywSb4wUot"
Access_token_secret = "jMnfk39glOFs1g9v7RLdlwWhqeDEvYeqBxAbvVxAkQZpU"

In [3]:
auth = tweepy.OAuthHandler(API_KEY, API_secret_key)
auth.set_access_token(Access_token, Access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

search_words = "#Arsenal OR "
date_since = "2019-01-01"

In [4]:
tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since).items(1000)

tweet_details = [[tweet.created_at, tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]
tweet_df = pd.DataFrame(data=tweet_details, columns=["date","text","user", "location"])
tweet_df.head(20)

,date,text,user,location
0,2021-01-15 08:37:48,@MixmanFIFA @DonkTrading this i can confirm,fifasimp69,
1,2021-01-15 08:37:48,RT @MsNitaTheBest: @chunkytingz The face //...,feet_blonde,
2,2021-01-15 08:37:48,RT @gtconway3d: Don't anyone tell them that th...,FaithMelody17,
3,2021-01-15 08:37:48,"RT @Handsome_Kev: You cannot fuck, drink, smok...",Queen_Kiaaaa,with my son
4,2021-01-15 08:37:48,"Take a Deep breath in and let it slowly out, t...",niceladyhelp,
5,2021-01-15 08:37:48,RT @nikkekeke: please help!\n\nplease block/re...,ustipcictete,Republic of Croatia
6,2021-01-15 08:37:48,RT @Asthete4: Let's take a stand together.\n#b...,zohrababarkhan1,
7,2021-01-15 08:37:48,RT @JasB_Lee: #Master Sad Story in North\n\n #...,AkashDani7,"Dindigul, India"
8,2021-01-15 08:37:48,RT @Whee_GlobalWind: [📸] Check out whee_inthem...,MoonSunrise12,
9,2021-01-15 08:37:48,"@ot4__blackpink bestie, I saw this on FB last ...",AAonTwt,she/her


In [5]:
tweet_df.tail(10)

,date,text,user,location
990,2021-01-15 08:37:47,RT @LavanyaBallal: The beat seller of the year...,d33p_crj,
991,2021-01-15 08:37:47,@NuZookeeper You will find only what you bring...,NuZoo4,
992,2021-01-15 08:37:47,RT @DarkVixen9: It is not a role\nIt is not a ...,eyemblacksheep,"Newcastle Upon Tyne, England"
993,2021-01-15 08:37:47,"Even if you were simply just in my circle,\n i...",MilzClayton,
994,2021-01-15 08:37:47,@vvhorenika HELP-,AlmondRodrigo,"Ava liked x4 , Iggy like x1"
995,2021-01-15 08:37:47,IAM comment section 😭\n\nLink: https://t.co/dT...,itshanbeom,
996,2021-01-15 08:37:47,@JanHoffmann3D Letter S,safelars,larsvoegerl@gmail.com
997,2021-01-15 08:37:47,@ZealousTheWolf Boyfriend!!!!!!!,mastros11,
998,2021-01-15 08:37:47,RT @dwoodru11: https://t.co/a0y4L3g7er ..... $...,Jcazz17,
999,2021-01-15 08:37:47,"@TigerHimbo I feel it bro, but AYE! You a whol...",MeMonstrous,Da Grotto


In [6]:
def clean_tweets(text):
  text = re.sub("RT @[\w]*:","",text)
  text = re.sub("@[\w]*","",text)
  text = re.sub("https?://[A-Za-z0-9./]*","",text)
  text = re.sub("\n","",text)
  text = re.sub('[(\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f)]+','',text)
  return text

tweet_df['text'] = tweet_df['text'].apply(lambda x: clean_tweets(x))

tweet_df.head(20)

,date,text,user,location
0,2021-01-15 08:37:48,this i can confirm,fifasimp69,
1,2021-01-15 08:37:48,The face /// The toes,feet_blonde,
2,2021-01-15 08:37:48,Don't anyone tell them that this would reduce...,FaithMelody17,
3,2021-01-15 08:37:48,"You cannot fuck, drink, smoke, shop or work y...",Queen_Kiaaaa,with my son
4,2021-01-15 08:37:48,"Take a Deep breath in and let it slowly out, t...",niceladyhelp,
5,2021-01-15 08:37:48,please help!please block/report for impersona...,ustipcictete,Republic of Croatia
6,2021-01-15 08:37:48,Let's take a stand together.#boycottoncampuse...,zohrababarkhan1,
7,2021-01-15 08:37:48,#Master Sad Story in North #Valimai,AkashDani7,"Dindigul, India"
8,2021-01-15 08:37:48,[] Check out whee_inthemood IG story update: ...,MoonSunrise12,
9,2021-01-15 08:37:48,"bestie, I saw this on FB last week I was like...",AAonTwt,she/her


In [7]:
tweet_df.to_csv('arsenal_tweets.csv', mode='a')

In [8]:
df = pd.read_csv("arsenal_tweets.csv")
df.head()

,Unnamed: 0,date,text,user,location
0,0.0,2021-01-13 08:51:43,"Aubameyang: ""From the beginning of this seaso...",andyfashions254,Nairobi
1,1.0,2021-01-13 08:50:20,#Chelsea Lose to #Arsenal make a comeback win ...,DX349,NaN
2,2.0,2021-01-13 08:49:40,Who were the last six Englishmen to win Arsena...,myfootballfacts,NaN
3,3.0,2021-01-13 08:49:10,One of the greatest midfielders ever .. and s...,samynteresse,Santiago Bernabeú
4,4.0,2021-01-13 08:48:49,Our number one 1⃣ #Arsenal #AFC #Leno,ssskyb,NaN


In [9]:
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
df['sentiment'] = df['text'].apply(lambda x: sid.polarity_scores(x))


def sentimentVerdict(sentiment):
  if sentiment['compound'] >= 0.05:
    return "Positive"
  elif sentiment['compound'] <= -0.05:
    return "Negative"
  else:
    return "Neutral"

In [11]:
df['sentiment_overall'] = df['sentiment'].apply(lambda x: sentimentVerdict(x))
df.head(20)

,Unnamed: 0,date,text,user,location,sentiment,sentiment_overall
0,0.0,2021-01-13 08:51:43,"Aubameyang: ""From the beginning of this seaso...",andyfashions254,Nairobi,"{'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'comp...",Negative
1,1.0,2021-01-13 08:50:20,#Chelsea Lose to #Arsenal make a comeback win ...,DX349,NaN,"{'neg': 0.223, 'neu': 0.62, 'pos': 0.157, 'com...",Negative
2,2.0,2021-01-13 08:49:40,Who were the last six Englishmen to win Arsena...,myfootballfacts,NaN,"{'neg': 0.0, 'neu': 0.657, 'pos': 0.343, 'comp...",Positive
3,3.0,2021-01-13 08:49:10,One of the greatest midfielders ever .. and s...,samynteresse,Santiago Bernabeú,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",Positive
4,4.0,2021-01-13 08:48:49,Our number one 1⃣ #Arsenal #AFC #Leno,ssskyb,NaN,"{'neg': 0.0, 'neu': 0.822, 'pos': 0.178, 'comp...",Positive
5,5.0,2021-01-13 08:47:47,"Aubameyang: ""From the beginning of this seaso...",Marilyn19031538,NaN,"{'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'comp...",Negative
6,6.0,2021-01-13 08:47:02,Which ten players appeared for Chelsea and the...,myfootballfacts,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Neutral
7,7.0,2021-01-13 08:43:33,Auba catching Laca on the massage table post ...,LeftyIrishBear,Equality Street,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Neutral
8,8.0,2021-01-13 08:42:25,Auba catching Laca on the massage table post ...,marrhtinz,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Neutral
9,9.0,2021-01-13 08:41:29,"Aubameyang: ""I always try to give my best, no...",ArsenalNLKingz,"London, England","{'neg': 0.0, 'neu': 0.902, 'pos': 0.098, 'comp...",Positive


In [12]:
df['sentiment']

0       {'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'comp...
1       {'neg': 0.223, 'neu': 0.62, 'pos': 0.157, 'com...
2       {'neg': 0.0, 'neu': 0.657, 'pos': 0.343, 'comp...
3       {'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...
4       {'neg': 0.0, 'neu': 0.822, 'pos': 0.178, 'comp...
                              ...                        
1996    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1997    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1998    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1999    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
2000    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
Name: sentiment, Length: 2001, dtype: object

In [13]:
df['sentiment_overall'].value_counts()

Positive    1004
Neutral      662
Negative     335
Name: sentiment_overall, dtype: int64